In [104]:
import os

import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Делаем пайморфи который приведет все к леммам и припишет POS

In [105]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

Загружаем модель семантики

In [106]:
m = 'ruwikiruscorpora_0_300_20.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

In [36]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            with open(open_name, 'r', encoding='utf-8') as f:
                text = f.read()
                yield text

Обходим тексты и для каждого считаем средний вектор из векторов слов

In [60]:
anekdots_texts = []
for text in read_txt('anekdots'):
    text_vectors = []
    words = text.split()
    for word in words:
        norm_word = normalize_and_add_pos(word)
        if norm_word in model:
            text_vectors.append(model[norm_word])
    mean_vector = sum(text_vectors) / len(text_vectors)
    anekdots_texts.append(mean_vector)

In [64]:
izvest_texts = []
for text in read_txt('izvest'):
    text_vectors = []
    words = text.split()
    for word in words:
        norm_word = normalize_and_add_pos(word)
        if norm_word in model:
            text_vectors.append(model[norm_word])
    mean_vector = sum(text_vectors) / len(text_vectors)
    izvest_texts.append(mean_vector)

In [66]:
teh_mol_texts = []
for text in read_txt('teh_mol'):
    text_vectors = []
    words = text.split()
    for word in words:
        norm_word = normalize_and_add_pos(word)
        if norm_word in model:
            text_vectors.append(model[norm_word])
    mean_vector = sum(text_vectors) / len(text_vectors)
    teh_mol_texts.append(mean_vector)

Делаем датафрейм

In [77]:
data = [vec for vec in anekdots_texts] + [vec for vec in izvest_texts] + [vec for vec in teh_mol_texts]
df = pd.DataFrame.from_records(data)

In [85]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,class
0,-0.005327,-0.002550,-0.012691,0.015691,0.009480,-0.000163,0.007628,-0.023825,0.010015,0.004410,...,0.003710,0.023587,0.014549,0.002639,0.012199,0.020589,0.017731,-0.013880,-0.003637,anekdot
1,-0.000636,-0.022139,0.006247,0.015691,0.007062,0.021689,0.003270,-0.011606,0.003766,0.015540,...,0.045119,0.013040,-0.001982,0.008188,0.020509,0.004625,0.045025,-0.031724,-0.003661,anekdot
2,-0.020983,-0.024008,0.007685,0.018271,0.007318,0.033143,0.023785,-0.006739,-0.007860,-0.002580,...,0.045904,0.012120,-0.019283,0.015308,-0.005188,-0.009700,0.009690,-0.025878,-0.007407,anekdot
3,-0.004453,-0.028444,0.007522,0.014499,0.020345,0.012675,0.006424,-0.008916,-0.002845,0.025336,...,0.019075,0.016369,-0.000082,0.012354,0.016641,-0.002264,0.013714,-0.009071,-0.006429,anekdot
4,0.017546,-0.029710,0.014734,0.006084,0.031617,0.020996,0.016895,-0.018471,-0.028365,-0.007889,...,0.066247,0.033964,-0.003239,0.056341,-0.000883,-0.006328,0.007141,-0.042952,0.004031,anekdot


In [83]:
df['class'] = ['anekdot' for _ in range(125)] + ['izvest' for _ in range(125)] + ['teh_mol' for _ in range(125)]

Делим на трейн и тест

In [93]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:300], df['class'], test_size=0.2)

Обучаем форест

In [102]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a = accuracy_score(y_test, y_pred)
f = f1_score(y_test, y_pred, average=None)
print('{:>15}: {}\n'.format('accuracy', a))
print('{:>15}:'.format('f_scores'))
for cl, f_cl in zip(clf.classes_, f):
    print('{:>15}: {}'.format(cl, f_cl))

       accuracy: 0.9466666666666667

       f_scores:
        anekdot: 0.9811320754716981
         izvest: 0.9361702127659574
        teh_mol: 0.9199999999999999


Результат, такой неплохой